#Tool for running NL-Bayes denoising on entire directories. This implementation ONLY works on Unix systems. If not using Colab, simply change the first cell to point at the comparable directories on your system. 

##Uses the Lebrun, Buades, and Morel c++ implementation found on IPOL 


## Converting images to png

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Lab/Grain-U-Net'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/MyDrive/{}/src'.format(FOLDERNAME))

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/$FOLDERNAME

/content/drive/My Drive/Lab/Grain-U-Net


In [4]:
#first, make save dirs

import os
from pathlib import Path
from pathlib import PurePath

test_dir = Path('data/test_Pt_Axon_3')
out_dir = Path('data/test_Pt_Axon_3_png')

os.mkdir(out_dir)

for FOV in test_dir.glob('*'):
    if not (FOV / 'raw').is_dir():
        continue
    
    new_dir = (out_dir / PurePath(FOV).name)
    if new_dir.is_dir():
        continue
    
    os.mkdir(new_dir)
    os.mkdir(new_dir / 'raw')
    

FileExistsError: ignored

In [5]:
#reading in images and converting to png + cropping

from pathlib import Path
from skimage import io
import numpy as np

for FOV in test_dir.glob('*'):
    if not (FOV / 'raw').is_dir():
        continue
    for image in FOV.glob('raw/*.tiff'):
        img = io.imread(image)
        img = img[124:1924, 124:1924]
        save_path = out_dir / PurePath(FOV).name / 'raw' / (str(PurePath(image).name).split('.')[0] + '.png')
        io.imsave(save_path, (img))
        

## Running denoising

In [ ]:
#more  save dirs

import os
from pathlib import Path
from pathlib import PurePath

noisy_dir = Path('data/test_Pt_Axon_Seq_png')
clean_dir = Path('data/test_Pt_Axon_Seq_denoised')

os.mkdir(clean_dir)

for FOV in noisy_dir.glob('*'):
    if not (FOV / 'raw').is_dir():
        continue
    
    new_dir = (clean_dir / PurePath(FOV).name)
    if new_dir.is_dir():
        continue
    
    os.mkdir(new_dir)
    os.mkdir(new_dir / 'raw')

In [ ]:
#give needed permissions and compile code

!chmod +x ./src/nl-bayes_20130617/NL_Bayes
%cd src/nl-bayes_20130617
!make
%cd /content/drive/My\ Drive/$FOLDERNAME

/content/drive/MyDrive/Lab/Grain-U-Net/src/nl-bayes_20130617
make: 'NL_Bayes' is up to date.
/content/drive/My Drive/Lab/Grain-U-Net


In [ ]:
#using subprocess to run the c++ code through command line. If a faster/more elegant/more accessible solution is desired, recommend a boost python implementation

import subprocess

subprocess.run(['sh', '-c', './src/nl-bayes_20130617/NL_Bayes'], capture_output=True)

noise = 30

for FOV in noisy_dir.glob('*'):
    if not (FOV / 'raw').is_dir(): 
        continue
    
    for image in FOV.glob('raw/*.png'):
        save_dir = clean_dir / PurePath(FOV).name / 'raw' / PurePath(image).name
        program = './src/nl-bayes_20130617/NL_Bayes'
        arg = ' {} {} 0 noisy.png {} basic.png difference.png bias.png biasbasic.png diffbias.png 1 0 0'.format(image, str(noise), save_dir)
        subprocess.run(['sh', '-c', program + arg])
        print('Finished image' + str(PurePath(image).name))

Finished image9.png
Finished image11.png
Finished image10.png
Finished image10.png
Finished image9.png
Finished image8.png
Finished image7.png
Finished image9.png
Finished image8.png
Finished image8.png
Finished image7.png
Finished image6.png
Finished image5.png
Finished image7.png
Finished image6.png
Finished image5.png
Finished image4.png
Finished image6.png
Finished image3.png
Finished image5.png
Finished image4.png
Finished image3.png
Finished image2.png
Finished image4.png
Finished image1.png
Finished image3.png
Finished image2.png
Finished image1.png
Finished image0.png
Finished image2.png


In [ ]:
!rm basic.png
!rm difference.png
!rm mea

rm: cannot remove 'mea': No such file or directory
